<a href="https://colab.research.google.com/github/PeaBrane/rbm_jax/blob/main/rbm_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training an RBM using jax

This notebook serves as a minimal example for training an RBM (restricted Boltzmann machine) using the ML framework, jax. It is designed to be a minimal introduction to the concept of unsupervised learning using stochastic neural networks, and also a proof-of-concept for the simplicity and power of the jax framework (in particular its jit compiler).

## Import ML libraries

First, we import all the necessary libraries from numpy, jax, and tensorflow. Note that these libraries are included in the Google Colab runtime environment. To enable efficient training with jax, make sure to enable the GPU accelerator in the runtime type if using Colab.

In [8]:
import numpy as np
from numpy.random import rand, permutation

import jax
import jax.numpy as jnp
from jax.scipy.special import logit, expit

import tensorflow as tf

## Load the MNIST dataset

Next, we download the MNIST dataset as numpy arrays from Keras, and rescale the image greyscales to range from 0 to 1.

In [2]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, x_train.shape[1]*x_train.shape[2]).astype('float32') / 256  # [data_size, n_visible]
x_test = x_test.reshape(-1, x_test.shape[1]*x_test.shape[2]).astype('float32') / 256 # [data_size, n_visible]

print(x_train.shape, x_test.shape)

(60000, 784) (10000, 784)


Before doing any unsupervised learning, let's first try to train a simple linear classifier on the dataset, so we have some baseline accuracy for reference later on. We should reach slightly above 90%.

In [3]:
def accuracies(dataset, epochs, batch_size):
  x_train, y_train, x_test, y_test = dataset

  linear_classifier = tf.keras.Sequential(tf.keras.layers.Dense(10))
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction="auto", name='loss')
  acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

  linear_classifier.compile(optimizer='adam', loss=loss, metrics=acc)
  history = linear_classifier.fit(x_train, y_train, batch_size, epochs, validation_data=(x_test, y_test), verbose=0)

  return history.history['accuracy'][-1], history.history['val_accuracy'][-1]


train_acc, val_acc = accuracies((x_train, y_train, x_test, y_test), epochs=5, batch_size=256)

print(f'train_acc: {train_acc:.3f}, val_acc: {val_acc:.3f}')

train_acc: 0.914, val_acc: 0.917


## Performing unsupervised learning

For most unsupervised learning problems without any labels, it is exponentially difficult to obtain the exact gradients of the weights (which requires iterating through all possible visible configurations). Instead, we approximate the gradients of the weights from statistics gathered by a Gibbs chain starting from the dataset, and conditionally sampling the visible/hidden activations in the RBM back and forth. 

This form of stochastic gradient update using a Gibbs chain is called contrastive divergence, or CD for short. This [set of notes](https://christian-igel.github.io/paper/TRBMAI.pdf) provides a detailed description of the CD algorithm. An update is called CD-k if the Gibbs chain passed through the RBM k times. We now try to implement efficiently in jax the standard CD-k algorithm.

In [4]:
def contrastive_divergence(inputs, weights, biases_visible, biases_hidden, iterations=1, lr=1e-2):
  n_ensemble, batch_size, n_visible = inputs.shape
  n_hidden = weights.shape[-1]

  # initialize the visible layer as the input data, and the hidden layer as zeros
  visible = inputs
  hidden = jnp.zeros([n_ensemble, batch_size, n_hidden])

  # pre-compute the random tensor used to sampling the neural activations later
  rand_hidden = logit(rand(iterations, n_ensemble, batch_size, n_hidden))
  rand_visible = logit(rand(iterations, n_ensemble, batch_size, n_visible))

  # compute the exact gradient statistics conditioned on the dataset; this is the "data term" or "positive gradient"
  grads_visible_data = visible
  grads_hidden_data = jax.nn.sigmoid(jnp.einsum('cbi,cij->cbj', visible, weights) + biases_hidden[:, jnp.newaxis, ...])
  grads_weights_data = jnp.einsum('cbi,cbj->cij', visible, grads_hidden_data) / batch_size
  grads_visible_data, grads_hidden_data = grads_visible_data.mean(1), grads_hidden_data.mean(1)

  # pre-allocate the memory for aggregating the gradient statistics gathered by the Gibbs chain
  grads_weights = jnp.zeros([n_ensemble, n_visible, n_hidden])
  grads_visible = jnp.zeros([n_ensemble, n_visible])
  grads_hidden = jnp.zeros([n_ensemble, n_hidden])

  # iterate the Gibbs chain
  for iteration in jnp.arange(iterations):
    # sample the hidden and visible activations
    hidden = (jnp.einsum('cbi,cij->cbj', visible, weights) + biases_hidden[:, np.newaxis, :] > rand_hidden[iteration, ...]).astype('float32')
    visible = (jnp.einsum('cbj,cij->cbi', hidden, weights) + biases_visible[:, np.newaxis, :] > rand_visible[iteration, ...]).astype('float32')

    # aggregate the gradient statistics from the Gibbs chain; this is the "model term" or "negative gradient"
    grads_weights += jnp.einsum('cbi,cbj->cij', visible, hidden) / batch_size
    grads_hidden += hidden.mean(1)
    grads_visible += visible.mean(1)

  # update and return the weights and biases
  weights += lr*(grads_weights_data - grads_weights / iterations)
  biases_visible += lr*(grads_visible_data - grads_visible / iterations)
  biases_hidden += lr*(grads_hidden_data - grads_hidden / iterations) 
  return weights, biases_visible, biases_hidden


# we use jit (just-in-time) to compile the CD algorithm in jax, to further improve efficiency
cd_jitted = jax.jit(contrastive_divergence, static_argnums=[4, 5])

Now, let's initialize multiple independent RBMs and train them using CD at the same time. This may take a while (several minutes), so make yourself a cup of coffee.

In [5]:
n_ensemble = 5  # number of independent RBMs to train
epochs = 50
iterations = 3  # iterations for contrastive divergence
batch_size = 100
learning_rates = np.geomspace(1e-1, 1e-3, epochs)

data_size = x_train.shape[0]
n_visible = x_train.shape[1]
n_hidden = 128  # number of hidden neurons in the RBM

# weight initialization (appropriately normalized)
weights = jnp.array((-1 + 2*rand(n_ensemble, n_visible, n_hidden)) / np.sqrt(n_visible))
biases_visible = jnp.array((-1 + 2*rand(n_ensemble, n_visible)) / np.sqrt(n_visible))
biases_hidden = jnp.array((-1 + 2*rand(n_ensemble, n_hidden)) / np.sqrt(n_visible))

for epoch, lr in zip(range(epochs), learning_rates):
  # randomly shuffle the dataset and divide into batches for each training ensemble
  permutes = np.vstack([permutation(data_size) for _ in range(n_ensemble)])
  batches = np.split(x_train.take(permutes, axis=0), round(data_size / batch_size), axis=1)  # [ensembles, batch_size, n_visible] * batches

  # run the CD algorithm batch-wise
  for batch in batches:
    batch = jnp.array(batch, dtype='float32')
    weights, biases_visible, biases_hidden = cd_jitted(batch, weights, biases_visible, biases_hidden, iterations, lr)

## Inspecting the RBM

Now we can evaluate the RBMs we trained by attaching a linear classifier to the hidden layer, and see if the classifier does better on the features implicitly extracted by the RBMs. Note that the RBM weights are frozen so they don't see the labels.

Turns out the accuracies did not improve by much compared to our previously established baseline (or attaching a linear classifier directly to the input data)...

In [6]:
x_train_hidden = np.array(jnp.einsum('bi,cij->cbj', jnp.array(x_train), weights) + biases_hidden[:, np.newaxis, :])
x_test_hidden = np.array(jnp.einsum('bi,cij->cbj', jnp.array(x_test), weights) + biases_hidden[:, np.newaxis, :])

accs = [accuracies((x_train_hidden[ensemble], y_train, x_test_hidden[ensemble], y_test), epochs=5, batch_size=256) for ensemble in range(n_ensemble)]
print('(train_acc, test_acc)')
accs

(train_acc, test_acc)


[(0.8892833590507507, 0.8974999785423279),
 (0.8916333317756653, 0.9014999866485596),
 (0.8856666684150696, 0.8944000005722046),
 (0.8928999900817871, 0.8996000289916992),
 (0.8932166695594788, 0.8982999920845032)]

However, there is one cool feature of the RBM we have not explored yet, and that is its generative nature. In short, we can try to "interpolate" between the input images, by performing a linear interpolating of the sampled hidden activations, and then passing these interpolations back to the visible layer to see how they look.

In particular, if the RBM was able to extract non-trivial features in the hidden layer, then the input images will appear to gradually "morph" into each other, instead of being a trivial pixel-wise intensity interpolation between images. Let's check it out.

In [13]:
import time
from PIL import Image
from IPython.display import clear_output


n_images = 5  # number of images to interpolate between
n_interps = 40  # number of interpolation frames between two images

# sample the hidden activations with the trained RBMs
x_slice = jnp.array(x_test[:n_images], dtype='float32')
y_slice = jnp.einsum('bi,cij->bcj', x_slice, weights) + biases_hidden
y_slice = (y_slice > logit(rand(n_images, n_ensemble, n_hidden))).astype('float32')

y_slice = jnp.stack([y_slice[:-1, ...], y_slice[1:, ...]], axis=0).repeat(n_interps, axis=1)
interp_ratios = jnp.linspace(0, 1, n_interps)
y_interps = y_slice * \
  jnp.tile(jnp.stack([1-interp_ratios, interp_ratios], axis=0), [1, n_images-1])[..., jnp.newaxis, jnp.newaxis]
y_interps = y_interps.sum(0)  # [n_frames, n_ensemble, image_size]

x_interps = jnp.einsum('bcj,cij->bci', y_interps, weights) + biases_visible
x_interps = expit(x_interps).reshape(-1, n_ensemble, 28, 28)  # [n_frames, n_ensemble, width, height]
x_interps = np.array(x_interps)

for x_interp in x_interps:
  x_interp = np.concatenate([x_instance for x_instance in x_interp], axis=1)
  imgs = Image.fromarray((x_interp * 256).clip(0, 255).astype('uint8'), 'L')
  clear_output(wait=True)
  display(imgs)
  time.sleep(0.2)